In [59]:
import json
import time
from joblib import load
from tqdm import tqdm
import numpy as np
import pandas as pd
from joblib import dump
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, r2_score
from sklearn import tree
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import os
import sys
sys.path.insert(0, '..')
np.set_printoptions(threshold=sys.maxsize, edgeitems=30, linewidth=100000, precision=2)

In [60]:
def create_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def sample_grouped_train_data(train_input, train_output, train_fraction, group_size, random_state=42):

    n_total_groups = len(train_input) // group_size
    n_sample_groups = int(train_fraction * n_total_groups)

    np.random.seed(random_state)
    selected_group_indices = np.random.choice(n_total_groups, size=n_sample_groups, replace=False)

    selected_row_indices = []
    for group_idx in selected_group_indices:
        start_idx = group_idx * group_size
        selected_row_indices.extend(range(start_idx, start_idx + group_size))

    train_input_sampled = train_input[selected_row_indices]
    train_output_sampled = train_output[selected_row_indices]

    return train_input_sampled, train_output_sampled

In [61]:
current_dir = os.getcwd()
config_file_path = os.path.join(current_dir, 'config_regression.json')

with open(config_file_path, 'r', encoding='utf-8') as file:
    config = json.load(file)

In [62]:
# Load training and testing data
training_data = pd.read_csv(config['training_data_path'], sep=',')
testing_data = pd.read_csv(config['testing_data_path'], sep=',')

# Prepare feature names
des_var_names = config['design_variable_names']
inp_var_names = des_var_names + ['strain']
feature_names = inp_var_names
n_inp = len(inp_var_names)

x_train = training_data[inp_var_names].to_numpy().reshape(-1, n_inp)
y_train = training_data['stress'].to_numpy().flatten()

# Prepare input/output for training
# x_train = training_data[inp_var_names].to_numpy().reshape(-1, n_inp)
# y_train = training_data['stress'].to_numpy().flatten()

# Prepare input/output for testing
x_test = testing_data[inp_var_names].to_numpy().reshape(-1, n_inp)
y_test = testing_data['stress'].to_numpy().flatten()

# Output dataset size
print('# designs for training:', x_train.shape[0])
print('# designs for testing:', x_test.shape[0])

# Create result folder
folder_name = 'results_rf_regression'
create_dir(folder_name)

# Save design variable bounds from training set
des_var_bounds = np.vstack([np.min(x_train[:,:-1], axis=0), np.max(x_train[:,:-1], axis=0)]).T
np.save(f'{folder_name}/des_var_bounds.npy', des_var_bounds)

# designs for training: 61860
# designs for testing: 6930


# Hyperparameter tuning

In [63]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import pandas as pd
from joblib import dump

depths = [10, 12, 15, 20, 25, 50]
train_mape_list = []
test_mape_list = []
train_mse_list = []
test_mse_list = []

for d in depths:
    model = RandomForestRegressor(
        n_estimators=config['num_estimators'],
        criterion='squared_error',
        max_depth=d,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=0
    )
    model.fit(x_train, y_train)

    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # MAPE
    mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

    # MSE
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)

    train_mape_list.append(mape_train)
    test_mape_list.append(mape_test)
    train_mse_list.append(mse_train)
    test_mse_list.append(mse_test)

    print(f"max_depth={d}, "
          f"Train MAPE={mape_train:.4f}, Test MAPE={mape_test:.4f}, "
          f"Train MSE={mse_train:.4f}, Test MSE={mse_test:.4f}")

    # dump(model, f'{folder_name}/forest_depth_{d}.joblib')

results_df = pd.DataFrame({
    "max_depth": depths,
    "train_mape": train_mape_list,
    "test_mape": test_mape_list,
    "train_mse": train_mse_list,
    "test_mse": test_mse_list
})
results_df.to_csv("rf_depth.csv", index=False)
print("Tree depth tuning result saved to rf_depth.csv")

max_depth=10, Train MAPE=0.0867, Test MAPE=0.0978, Train MSE=0.0003, Test MSE=0.0004
max_depth=12, Train MAPE=0.0513, Test MAPE=0.0654, Train MSE=0.0001, Test MSE=0.0002
max_depth=15, Train MAPE=0.0208, Test MAPE=0.0403, Train MSE=0.0000, Test MSE=0.0001
max_depth=20, Train MAPE=0.0104, Test MAPE=0.0350, Train MSE=0.0000, Test MSE=0.0001
max_depth=25, Train MAPE=0.0101, Test MAPE=0.0349, Train MSE=0.0000, Test MSE=0.0001
max_depth=50, Train MAPE=0.0101, Test MAPE=0.0349, Train MSE=0.0000, Test MSE=0.0001
max_depth=100, Train MAPE=0.0101, Test MAPE=0.0349, Train MSE=0.0000, Test MSE=0.0001
Tree depth tuning result saved to rf_depth.csv


# Data efficiency check

In [64]:
# class_weight='balanced'
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [65]:
# Record results
data_efficiency_results = []

# Sweep over different training fractions
for train_fraction in np.linspace(0.1, 1, 10):
    # training_data_sampled = training_data.sample(frac=train_fraction, random_state=42)
    # x_train = training_data_sampled[inp_var_names].to_numpy().reshape(-1, n_inp)
    # y_train = training_data_sampled['stress'].to_numpy().flatten()

    x_train_sampled, y_train_sampled = sample_grouped_train_data(x_train, y_train, train_fraction, group_size=30, random_state=42)

    # Save bounds for the current training set (optional)
    des_var_bounds = np.vstack([np.min(x_train_sampled[:, :-1], axis=0), np.max(x_train_sampled[:, :-1], axis=0)]).T
    np.save(f'{folder_name}/des_var_bounds_{int(train_fraction*100)}.npy', des_var_bounds)

    # Train model
    reg = RandomForestRegressor(n_estimators=config['num_estimators'],
                                 criterion='squared_error',
                                 max_depth=15,
                                 min_samples_split=2,
                                 min_samples_leaf=1,
                                 random_state=42)
    reg.fit(x_train_sampled, y_train_sampled)

    # Evaluate on train
    y_pred_train_sampled = reg.predict(x_train_sampled)
    mse_train = mean_squared_error(y_train_sampled, y_pred_train_sampled)
    mae_train = mean_absolute_error(y_train_sampled, y_pred_train_sampled)
    mape_train = mean_absolute_percentage_error(y_train_sampled, y_pred_train_sampled)
    # Evaluate on test
    y_pred_test = reg.predict(x_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

    # Store results
    data_efficiency_results.append({
        'fraction': train_fraction,
        'mse_train': mse_train,
        'mae_train': mae_train,
        'mape_train': mape_train,
        'mse_test': mse_test,
        'mae_test': mae_test,
        'mape_test': mape_test,
        'n_train': len(x_train_sampled)
    })

    dump(reg, f'{folder_name}/forest_{int(train_fraction*100)}.joblib')


In [10]:

results_df = pd.DataFrame(data_efficiency_results)
results_df.to_csv("results_summary.csv", index=False)

# Final model

In [66]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, r2_score

# Train a random forest
start_time = time.time()
reg = RandomForestRegressor(n_estimators=config['num_estimators'], criterion='squared_error',
                            max_depth=15, min_samples_split=2, min_samples_leaf=1, random_state=42)

final_fraction = 0.4
x_train_sampled, y_train_sampled = sample_grouped_train_data(x_train, y_train, final_fraction, group_size=30, random_state=42)
reg = reg.fit(x_train_sampled, y_train_sampled)
print('Training time: {:.2f}s'.format(time.time()-start_time))

# Evaluate the model on training data
y_pred_train = reg.predict(x_train_sampled)
mse_train = mean_squared_error(y_train_sampled, y_pred_train)
mae_train = mean_absolute_error(y_train_sampled, y_pred_train)
mape_train = mean_absolute_percentage_error(y_train_sampled, y_pred_train)
r2_train = r2_score(y_train_sampled, y_pred_train)

# Evaluate the model on test data
y_pred_test = reg.predict(x_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

# Print results
print('Training MSE:', mse_train)
print('Training MAE:', mae_train)
print('Training MAPE:', mape_train)
print('Training R2:', r2_train)

print('Test MSE:', mse_test)
print('Test MAE:', mae_test)
print('Test MAPE:', mape_test)
print('Test R2:', r2_test)

# Save results to file
lines = [
    'Train MSE: {:.4f}'.format(mse_train),
    'Train MAE: {:.4f}'.format(mae_train),
    'Train MAPE: {:.4f}'.format(mape_train),
    'Train R2: {:.4f}'.format(r2_train),
    'Test MSE: {:.4f}'.format(mse_test),
    'Test MAE: {:.4f}'.format(mae_test),
    'Test MAPE: {:.4f}'.format(mape_test),
    'Test R2: {:.4f}'.format(r2_test)
]

with open(f'{folder_name}/forest_regression_acc.txt', 'w') as f:
    f.write('\n'.join(lines))

# Save the model
dump(reg, f'{folder_name}/forest.joblib')



Training time: 3.01s
Training MSE: 1.490745000340074e-05
Training MAE: 0.0028037886643695223
Training MAPE: 0.018725340318051478
Training R2: 0.9994182203265911
Test MSE: 0.00015600136674686878
Test MAE: 0.008629479701011359
Test MAPE: 0.05699459797577057
Test R2: 0.9931315309820675


['results_rf_regression/forest.joblib']

In [67]:
used_train_data = np.hstack([x_train_sampled, y_train_sampled.reshape(-1, 1)])
col_names = inp_var_names + ["stress"]
used_train_df = pd.DataFrame(used_train_data, columns=col_names)
used_train_df.to_csv(f"stress_strain_training_data_fraction_{int(final_fraction*100)}.csv", index=False)

In [70]:
import pandas as pd


used_dataset = pd.read_csv(f"stress_strain_training_data_fraction_{int(final_fraction*100)}.csv")
used_dataset["strain_idx"] = used_dataset.groupby(['L', 'w', 'alpha', 'num_vert', 'num_hori']).cumcount()

used_dataset_flatten = used_dataset.pivot(
    index=['L', 'w', 'alpha', 'num_vert', 'num_hori'],
    columns='strain_idx',
    values='stress'
).reset_index()


n_stress = used_dataset_flatten.shape[1] - 5  

stress_cols = [f"s{i}" for i in range(2, n_stress + 2)]
used_dataset_flatten.columns = ['L', 'w', 'alpha', 'num_vert', 'num_hori'] + stress_cols

used_dataset_flatten.insert(5, "s1", 0)

param_cols = ['L', 'w', 'alpha', 'num_vert', 'num_hori']
stress_cols = [f"s{i}" for i in range(1, n_stress + 2)]
used_dataset_flatten = used_dataset_flatten[param_cols + stress_cols]

used_dataset_flatten.to_excel(f'train_fraction_{int(final_fraction*100)}.xlsx', index=False)

print("✅ recovered:", used_dataset_flatten.shape)


✅ recovered: (824, 36)


## Plot decision tree

Take a lot of time when tree is large

In [10]:
# from sklearn.tree import export_graphviz
# import graphviz
# from IPython.display import display

# tree = reg.estimators_[2]

# dot_data = export_graphviz(tree, out_file=None, 
#                            feature_names=inp_var_names,        
#                            filled=True, rounded=True, 
#                            special_characters=True)  

# graph = graphviz.Source(dot_data)  

# graph.render("decision_tree", format="pdf", cleanup=True)  

# display(graph)
